# Exercise 1: ETL with Spark

## What we will cover:
- Update Group Parameters
- Extract data from SQL
- Transform the Currencies to Euros
- Load Data to Delta tables
- Time Travel with Delta Tables

<div class="alert alert-block alert-danger">
<b>Important</b> Make sure you chose PySpark for your kernel
</div>

## Spark Architecture
![image.png](./images/exercise1/spark_archi.PNG)

## Apache Spark Architecture: Driver and Executors

### Cluster Manager:
- Manages resources across the cluster, allocating Executors to Spark applications.
- Examples include Apache Mesos, Apache Hadoop YARN, and Spark's standalone cluster manager.
- Responsible for monitoring and scheduling tasks on worker nodes.

### Driver:
- Centralized control unit of a Spark application.
- Runs the main() function and creates a SparkContext.
- Manages the execution of the Spark program.
- Coordinates tasks and schedules their execution on Executors.
- Keeps track of the overall application state.
- Communicates with the cluster manager to acquire and allocate resources.

### Executors:
- Worker nodes that perform the actual data processing tasks.
- Assigned tasks by the Driver for parallel execution.
- Responsible for running the individual stages of a Spark application.
- Manage and cache data in-memory during computation for efficient processing.
- Communicate with each other and the Driver for task coordination.
- Executors are launched on worker nodes by the cluster manager.

# 1. Setup Interactive Session

### We will use predefined commands for this Notebook called magic command to setup the Spark environment as below 

In [1]:
%manage_spark

MagicsControllerWidget(children=(Tab(children=(ManageSessionWidget(children=(HTML(value='<br/>'), HTML(value='…

1. Run the `%manage_spark` magic command.
2. Clic on `Add Endpoint`
3. Leave the settings as-is and clic on `Add endpoint` on the right

![image.png](./images/exercise1/init_spark.PNG)

1. Clic on `Create Session`
2. Your endpoint should appear in `Endpoint`
3. Update the `Name` to your group name
4. Change the `Language` to `Python`
5. Clic on `Create Session` on the right side

![image.png](./images/exercise1/init_spark2.PNG)

## Wait few minutes for the session to start.

### Meanwhile you can check the status here

1. Go back Unified Analytics
2. On the menu, select `Spark Interactive Sessions`

![image.png](./images/exercise1/menu.PNG)

3. Check the status of your session: it should take 2-3 minutes to start. When it's `Idle` go back to your `Notebook`

![image.png](./images/exercise1/session.PNG)

4. In the Notebook cell, the session should now be visible as `Idle` too. 

![image.png](./images/exercise1/session2.PNG)

In [2]:
%config_spark

Button(description='Submit', style=ButtonStyle())

Output()

<div class="alert alert-block alert-danger">
<b>Important</b> Do not restart the kernel
</div>

1. Run the `%config_spark` magic command.
2. Unless you want a specific configuration, leave the settings as-is and clic on `Submit` at the bottom

![image.png](./images/exercise1/config.PNG)

### Set up the group parameter

In [3]:
# Set your group name
group_name = "user2"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Check if GROUP is empty
if not group_name:
    print("Please set your group name before proceeding to the next cell.")
    # You can also raise an exception to stop execution if needed
    # raise ValueError("Group name is empty. Please set a valid group name.")
else:
    print("Group name is set. Proceed to the next cell.")
    # Your code for the next cell can go here

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Group name is set. Proceed to the next cell.

### Double Checking

In [5]:
# Check if GROUP is empty
if not group_name:
    print("Please set your group name before proceeding to the next cell.")
    # You can also raise an exception to stop execution if needed
    # raise ValueError("Group name is empty. Please set a valid group name.")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Import required libraries

In [6]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import udf, col, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import os

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Set up the Spark session
Adding delta extensions to the configuration to be able to interact with the delta tables

In [7]:
# Set up the Spark session
spark = SparkSession.builder \
    .appName("DataCleaningWithSpark") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

print("Pyspark session started")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Pyspark session started

# 2. Generate the Data in a CSV and extract it in a DataFrame

## Generate the Data

In [9]:
%run create_csv.py -c "Germany" -cu EUR -s 5 -sy 2019 -ey 2023 -csv "germany_sales_data_2019_2023.csv"

/opt/conda/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning:

Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning



# Variables Selected
	- Country: Germany
	- Currency: EUR
	- Stores: 5
	 ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
	- Start: 2019
	- End: 2023
Rates:
{'EUR': 1, 'AED': 3.98, 'AFN': 77.3, 'ALL': 102.96, 'AMD': 429.88, 'ANG': 1.94, 'AOA': 916.45, 'ARS': 928.42, 'AUD': 1.66, 'AWG': 1.94, 'AZN': 1.85, 'BAM': 1.96, 'BBD': 2.17, 'BDT': 118.95, 'BGN': 1.96, 'BHD': 0.407, 'BIF': 3101.42, 'BMD': 1.08, 'BND': 1.46, 'BOB': 7.5, 'BRL': 5.4, 'BSD': 1.08, 'BTN': 90.34, 'BWP': 14.94, 'BYN': 3.54, 'BZD': 2.17, 'CAD': 1.47, 'CDF': 3031.1, 'CHF': 0.979, 'CLP': 1061.95, 'CNY': 7.84, 'COP': 4225.92, 'CRC': 545.85, 'CUP': 26.01, 'CVE': 110.27, 'CZK': 25.29, 'DJF': 192.59, 'DKK': 7.46, 'DOP': 64.15, 'DZD': 146.17, 'EGP': 51.8, 'ERN': 16.25, 'ETB': 61.75, 'FJD': 2.44, 'FKP': 0.858, 'FOK': 7.46, 'GBP': 0.858, 'GEL': 2.93, 'GGP': 0.858, 'GHS': 14.35, 'GIP': 0.858, 'GMD': 71.54, 'GNF': 9298.22, 'GTQ': 8.45, 'GYD': 227.13, 'HKD': 8.48, 'HNL': 26.77, 'HRK': 7.53, 'HTG': 144.53, 'HUF': 395.5, 'IDR': 

In [11]:
%run create_csv.py -c "Czech Republic" -cu CZK -s 5 -sy 2019 -ey 2023 -csv "czech_sales_data_2019_2023.csv"

# Variables Selected
	- Country: Czech Republic
	- Currency: CZK
	- Stores: 5
	 ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
	- Start: 2019
	- End: 2023
Rates:
{'EUR': 1, 'AED': 3.98, 'AFN': 77.3, 'ALL': 102.96, 'AMD': 429.88, 'ANG': 1.94, 'AOA': 916.45, 'ARS': 928.42, 'AUD': 1.66, 'AWG': 1.94, 'AZN': 1.85, 'BAM': 1.96, 'BBD': 2.17, 'BDT': 118.95, 'BGN': 1.96, 'BHD': 0.407, 'BIF': 3101.42, 'BMD': 1.08, 'BND': 1.46, 'BOB': 7.5, 'BRL': 5.4, 'BSD': 1.08, 'BTN': 90.34, 'BWP': 14.94, 'BYN': 3.54, 'BZD': 2.17, 'CAD': 1.47, 'CDF': 3031.1, 'CHF': 0.979, 'CLP': 1061.95, 'CNY': 7.84, 'COP': 4225.92, 'CRC': 545.85, 'CUP': 26.01, 'CVE': 110.27, 'CZK': 25.29, 'DJF': 192.59, 'DKK': 7.46, 'DOP': 64.15, 'DZD': 146.17, 'EGP': 51.8, 'ERN': 16.25, 'ETB': 61.75, 'FJD': 2.44, 'FKP': 0.858, 'FOK': 7.46, 'GBP': 0.858, 'GEL': 2.93, 'GGP': 0.858, 'GHS': 14.35, 'GIP': 0.858, 'GMD': 71.54, 'GNF': 9298.22, 'GTQ': 8.45, 'GYD': 227.13, 'HKD': 8.48, 'HNL': 26.77, 'HRK': 7.53, 'HTG': 144.53, 'HUF': 395.5, 

In [13]:
%run create_csv.py -c "Swiss" -cu CHF -s 5 -sy 2019 -ey 2023 -csv "swiss_sales_data_2019_2023.csv"

# Variables Selected
	- Country: Swiss
	- Currency: CHF
	- Stores: 5
	 ['Store A', 'Store B', 'Store C', 'Store D', 'Store E']
	- Start: 2019
	- End: 2023
Rates:
{'EUR': 1, 'AED': 3.98, 'AFN': 77.3, 'ALL': 102.96, 'AMD': 429.88, 'ANG': 1.94, 'AOA': 916.45, 'ARS': 928.42, 'AUD': 1.66, 'AWG': 1.94, 'AZN': 1.85, 'BAM': 1.96, 'BBD': 2.17, 'BDT': 118.95, 'BGN': 1.96, 'BHD': 0.407, 'BIF': 3101.42, 'BMD': 1.08, 'BND': 1.46, 'BOB': 7.5, 'BRL': 5.4, 'BSD': 1.08, 'BTN': 90.34, 'BWP': 14.94, 'BYN': 3.54, 'BZD': 2.17, 'CAD': 1.47, 'CDF': 3031.1, 'CHF': 0.979, 'CLP': 1061.95, 'CNY': 7.84, 'COP': 4225.92, 'CRC': 545.85, 'CUP': 26.01, 'CVE': 110.27, 'CZK': 25.29, 'DJF': 192.59, 'DKK': 7.46, 'DOP': 64.15, 'DZD': 146.17, 'EGP': 51.8, 'ERN': 16.25, 'ETB': 61.75, 'FJD': 2.44, 'FKP': 0.858, 'FOK': 7.46, 'GBP': 0.858, 'GEL': 2.93, 'GGP': 0.858, 'GHS': 14.35, 'GIP': 0.858, 'GMD': 71.54, 'GNF': 9298.22, 'GTQ': 8.45, 'GYD': 227.13, 'HKD': 8.48, 'HNL': 26.77, 'HRK': 7.53, 'HTG': 144.53, 'HUF': 395.5, 'IDR': 17

## Verify if the Data is there

In [14]:
# Define the directory path
data_path = "file:///mounts/shared-volume/user/data"

# List files in the directory
files = spark.sparkContext.wholeTextFiles(data_path)

# Display the list of files
for file_path, _ in files.collect():
    print(file_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

file:///mounts/shared-volume/user/data/czech_sales_data_2019_2023.csv
file:///mounts/shared-volume/user/data/germany_sales_data_2019_2023.csv
file:///mounts/shared-volume/user/data/swiss_sales_data_2019_2023.csv

## Define a function to extract the data from CSV in a DataFrame

In [165]:
from pyspark.sql.types import IntegerType

def load_data(spark, country, data_path):
    # Define the path to the CSV file
    csv_path = f"{data_path}/{country}_sales_data_2019_2023.csv"

    # Define the schema with specific data types
    schema = StructType([
        StructField("PRODUCTID", IntegerType(), True),
        StructField("PRODUCT", StringType(), True),
        StructField("TYPE", StringType(), True),
        StructField("UNITPRICE", DoubleType(), True),
        StructField("UNIT", StringType(), True),
        StructField("QTY", IntegerType(), True),
        StructField("TOTALSALES", DoubleType(), True),
        StructField("CURRENCY", StringType(), True),
        StructField("STORE", StringType(), True),
        StructField("COUNTRY", StringType(), True),
        StructField("YEAR", IntegerType(), True)
    ])

    # Read data from the CSV file with the specified schema
    df = spark.read \
        .format("csv") \
        .schema(schema) \
        .option("header", "true") \
        .load(csv_path)

    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define a function to convert the currency to Euro

In [166]:
def clean_data(df, spark, country):
    # Define a UDF to convert currencies to EUR
    convert_udf = udf(lambda currency, amount: amount / CZK_TO_EUR_RATE if currency == "CZK" else amount / CHF_TO_EUR_RATE if currency == "CHF" else amount, DoubleType())

    # Apply the UDFs to the DataFrame
    corrected_df = df.withColumn("totalsales", convert_udf(col("currency"), col("totalsales"))) \
                     .withColumn("currency", lit("EUR"))

    # Show the results
    corrected_df.show()

    return corrected_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define a function to save the data to a Delta Tables 
<div class="alert alert-block alert-danger">
<b>Important</b> Make sure the path is correct
</div>

In [167]:
def write_data(df, country):
    delta_path = f"file:///mounts/shared-volume/shared/end2end-delta/{group_name}/{country}"

    # Check if the directory exists, and create it if it doesn't
    if not os.path.exists(delta_path):
        os.makedirs(delta_path)
        
    df.write.format("delta").mode("overwrite").save(delta_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Define the country names & currency rates 
<div class="alert alert-block alert-danger">
<b>Important</b> Make sure the values are correct
</div>

In [168]:
# Constants
COUNTRY_LIST = ["czech", "germany", "swiss"]
CZK_TO_EUR_RATE = 25
CHF_TO_EUR_RATE = 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Run the ETL for each countries

In [ ]:
for country in COUNTRY_LIST:
    # Load data from the DBs
    df = load_data(spark, country, data_path)
    df.show()
    
    # Clean the data
    cleaned_df = clean_data(df, spark, country)
    cleaned_df.printSchema()
    
    # Write the cleaned data back to the Delta Table
    write_data(cleaned_df, country)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Check if the Delta Tables were created

In [ ]:
for country in COUNTRY_LIST:
    # List files in a directory
    files = os.listdir(f"/mounts/shared-volume/shared/end2end-delta/{group_name}/{country}")
    print("Table:", country)
    
    for file in files:
        if file.endswith(".parquet"):
            full_path = os.path.join(f"/mounts/shared-volume/shared/end2end-delta/{group_name}/{country}", file)
            print("Saved in:", full_path)

    print()

## Time Travel

1. Create Initial Delta Table by loading the `czech` table:

In [ ]:
from delta.tables import DeltaTable
# Set the parameters
country = "czech"
delta_path = f"file:///mounts/shared-volume/shared/end2end-delta/{group_name}/{country}"

# Read the Delta table using the load method
read_df = spark.read.format("delta").load(delta_path)

# Show the contents of the DataFrame
read_df.show()

2. Overwrite the Delta Table with Selected Columns:

In [ ]:
# Select only a subset of columns for the initial Delta Table
selected_columns = ["type", "unitprice", "qty", "totalsales"]
select_delta_path = f"file:///mounts/shared-volume/shared/end2end-delta/{group_name}/{country}"

# Create the initial Delta Table with selected columns
df_select = df.select("type", "unitprice", "unit", "qty")
df_select.write.format("delta").mode("overwrite").save(select_delta_path)

3. Read the Delta Table with Selected Columns: the old columns are marked as `NULL`

In [ ]:
# Read the Delta table using the load method
read_df_select = spark.read.format("delta").load(delta_path)

# Show the contents of the DataFrame
read_df_select.show()

# Display the schema of the version 0 DataFrame
read_df_select.printSchema()

4. Display the versions available for this table:

In [ ]:
# Create a DeltaTable object
delta_table = DeltaTable.forPath(spark, delta_path)

# Get the history of the Delta table
history_df = delta_table.history()

# List all versions with timestamp
versions_with_timestamp = history_df.select("version", "timestamp").distinct().collect()

# Display the list of versions with timestamp
print("List of Delta Table Versions with Timestamp:")
for version_info in versions_with_timestamp:
    version = version_info["version"]
    timestamp = version_info["timestamp"]
    print(f"Version: {version}, Timestamp: {timestamp}")

5. Compare table content

In [ ]:
# Read a specific version (e.g., version 0) of the Delta table
read_df_version_0 = spark.read.format("delta").option("versionAsOf", "0").load(delta_path)
read_df_version_0.show()

# Read a specific version (e.g., version 1) of the Delta table
read_df_version_1 = spark.read.format("delta").option("versionAsOf", "2").load(delta_path)
read_df_version_1.show()

6. Rollback to Initial Delta Table

In [ ]:
# Read a specific version (e.g., version 0) of the Delta table
read_df_version_0 = spark.read.format("delta").option("versionAsOf", "0").load(delta_path)

# If you want to perform further actions or overwrite the current Delta table:
# Overwrite the current Delta table with version 0 data
read_df_version_0.write.format("delta").mode("overwrite").save(delta_path)

7. Read the Delta Table with Selected Columns: the old columns are back in the current version

In [ ]:
# Read the Delta table using the load method
read_df_select = spark.read.format("delta").load(delta_path)

# Show the contents of the DataFrame
read_df_select.show()

# Display the schema of the version 0 DataFrame
read_df_select.printSchema()

8. Display the versions available for this table:

In [ ]:
# Create a DeltaTable object
delta_table = DeltaTable.forPath(spark, delta_path)

# Get the history of the Delta table
history_df = delta_table.history()

# List all versions with timestamp
versions_with_timestamp = history_df.select("version", "timestamp").distinct().collect()

# Display the list of versions with timestamp
print("List of Delta Table Versions with Timestamp:")
for version_info in versions_with_timestamp:
    version = version_info["version"]
    timestamp = version_info["timestamp"]
    print(f"Version: {version}, Timestamp: {timestamp}")

## END